In [3]:
import pandas as pd
from openpyxl import load_workbook
from openpyxl.chart import ScatterChart, Reference, Series, marker
from openpyxl.utils import get_column_letter
from more_itertools import consecutive_groups
from statsmodels.formula.api import ols
from math import isnan

In [4]:
def get_compression_params():
    
    df = pd.read_excel('data/Corpos de prova.xlsx', sheet_name='Compressao', index_col=0)
    df = df.infer_objects()

    return df

def get_test_data(sample_id):
    file = 'data\Compressão Uniaxial\Curvas\%s.txt' % sample_id
    df = pd.read_csv(file, encoding='ANSI', sep='\t')

    df.columns = ['time', 'strain', 'force']

    return df

def new_sheet_from_template(workbook, template, sheet_name):
    ws = workbook.copy_worksheet(template)
    ws.title = sheet_name
    
    return ws

def filter_stress_interval(df, min_stress, max_stress):
    filtered = (df['stress'] > min_stress) & (df['stress'] < max_stress)

    return df[filtered]


def separate_loading_unloading(df):
    series = []

    for group in consecutive_groups(df.index):
        serie = df.loc[group]

        if (len(serie) > 3):
            series.append(serie)

    return series


def filter_series(series, intervals):
    new_series = []
    inicio = intervals[['c1_inicio', 'd1_inicio', 'c2_inicio',
                         'd2_inicio', 'c3_inicio', 'd3_inicio', 'c4_inicio']].values
    fim = intervals[['c1_fim', 'd1_fim', 'c2_fim',
                      'd2_fim', 'c3_fim', 'd3_fim', 'c4_fim']].values

    for i, serie in enumerate(series):
        if (not isnan(inicio[i]) and not isnan(fim[i])):
            serie = serie[int(inicio[i]): int(fim[i])]

        new_series.append(serie.reset_index())

    return new_series

def get_young_modulus(serie):
    formula = '%s ~ %s' % ('stress', 'spec_strain')
    result = ols(formula=formula, data=serie).fit()

    return result.params[1] / 1000

def set_def_start_to_zero(series):
    series = series.copy()
        
    for i, serie in enumerate(series):
        min_def = min(serie['spec_strain'])
        serie['spec_strain'] = serie['spec_strain'].apply(lambda val: val - min_def)
        
    return series
        


In [31]:
filename = "Young Modulus.xlsx"
wb = load_workbook('data/Young Modulus Template.xlsx')
template = wb['template']
summary = wb['summary']

compression_params = get_compression_params()

stages_first_row = 33
stage_n_cols = 5

modulus_row = 5
modulus_first_col = 2

chart_row = 8

summary_row = 2

for sample_id, params in compression_params.iterrows():
    
    # get test data for current sample
    test_data = get_test_data(sample_id)
    
    # create new worksheet
    ws = new_sheet_from_template(wb, template, sample_id) 

    # calc specific strain
    test_data['spec_strain'] = test_data['strain'] / params['comp_base']
    
    # calc stress
    test_data['stress'] = (test_data['force'] / params['area_circular']) / 10**6

    # get data between specified min and max stress
    test_data = filter_stress_interval(test_data, params['tensao_min'], params['tensao_max'])
    
    # separate loading and unloading stages
    series = separate_loading_unloading(test_data)
    
    # set all series to start with zero of specific strain
    series = set_def_start_to_zero(series)
    
    # filter series intervals for regression
    series = filter_series(series, params.iloc[6:])

    # start writing to xls
    ws['A2'] = sample_id
    ws['B2'] = params['area_circular']
    ws['C2'] = params['comp_base']
    ws['D2'] = params['resist_prev']
    ws['E2'] = params['tensao_min']
    ws['F2'] = params['tensao_max']

    summary_row = summary_row + 1
    cell = summary.cell(row=summary_row, column=1)
    cell.value = '=HYPERLINK("#\'%s\'!A1","%s")' % (sample_id, sample_id)
    
    for i, serie in enumerate(series):
        modulus = get_young_modulus(serie)
        ws.cell(row=modulus_row, column=modulus_first_col + i, value=modulus)
        
        
        first_col = (i * stage_n_cols) + 1
        for irow, data in serie.iterrows():
            row = stages_first_row + irow
            ws.cell(row=row, column=first_col, value=data['time'])
            ws.cell(row=row, column=first_col+1, value=data['strain'])
            ws.cell(row=row, column=first_col+2, value=data['spec_strain'])
            ws.cell(row=row, column=first_col+3, value=data['force'])
            ws.cell(row=row, column=first_col+4, value=data['stress'])
            
        chart = ScatterChart()
        chart.y_axis.title = 'Stress (MPa)'
        chart.x_axis.title = 'Specific Strain (mm/mm)'
        chart.height = 10.5
        chart.width = 21
        chart.legend = None
        chart.scatterStyle = 'marker'
        
        strain_ref = Reference(ws, min_col=first_col+2, min_row=stages_first_row, max_row=stages_first_row + len(serie))
        stress_ref = Reference(ws, min_col=first_col+4, min_row=stages_first_row, max_row=stages_first_row + len(serie))
        series = Series(stress_ref, strain_ref, title_from_data=False)
        series.marker= marker.Marker('x')
        series.graphicalProperties.line.noFill=True
        chart.series.append(series)
        anchor = '%s%s' % (get_column_letter(first_col), chart_row)
        ws.add_chart(chart, anchor)
        
    
wb.remove(template)
wb.active = summary
wb.save('data/%s' % filename)